In [15]:
import sys
import os
sys.path.append(os.path.abspath('../..'))
import datetime
import concurrent.futures
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import threading



In [ ]:
# # Now, from above, integrating partial scanning again!

# import sys
# import os
# import concurrent.futures
# sys.path.append(os.path.abspath('../..'))
# import pandas as pd
# import pyarrow as pa
# import pyarrow.parquet as pq
# from utlis.sync_utlis.sync_df_utlis import find_calib_file
# from utlis.scan_engine_utlis.scan_engine_utlis import (
#     read_failed_paths,
#     match_date_pattern,
#     assign_status_codes
# )
# from scan_engine.status_fields_config import STATUS_FIELDS_CONFIG

# # # Function to scan an individual folder (for parallel processing)
# # def scan_folder(folder_name, base_folder, failed_paths, config):
# #     folder_path = os.path.join(base_folder, folder_name)
# #     rec_files_data = []  # To store rec files and their status
# #     calib_files = []  # To store calibration files

# #     # Check for calibration files starting with 'calib'
# #     for file_name in os.listdir(folder_path):
# #         if file_name.startswith("calib"):
# #             calib_files.append(file_name)

# #     # Traverse subfolders within this folder
# #     for subfolder_name in os.listdir(folder_path):
# #         subfolder_path = os.path.join(folder_path, subfolder_name)

# #         # Check for subfolders starting with a digit (rec folders)
# #         if os.path.isdir(subfolder_path) and subfolder_name[0].isdigit():
# #             # Find calibration file for each subfolder
# #             calib_file = find_calib_file(subfolder_path)

# #             # Assign status codes dynamically based on the config
# #             rec_file_data = assign_status_codes(
# #                 folder_name, subfolder_path, calib_file, failed_paths, config
# #             )

# #             rec_file_data['rec_file'] = subfolder_name  # Add rec_file to the data
# #             rec_files_data.append(rec_file_data)

# #     return {
# #         'date_folder': folder_name,
# #         'calib_files': calib_files,  # Store the calibration files under date_folder level
# #         'rec_files_data': rec_files_data  # Each rec file with its status fields
# #     }

# # # Parallel version of the original log_folder_to_parquet function, with partial scan now.
# # # def log_folder_to_parquet(base_folder, parquet_file, failed_paths_file, config):
# # #     # Read manually inputted failed paths
# # #     failed_paths = read_failed_paths(failed_paths_file)

# # #     # Read the existing log to get already processed date folders
# # #     existing_df = pq.read_table(parquet_file).to_pandas()
# # #     logged_folders = existing_df['date_folder'].unique() if not existing_df.empty else []

# # #     # Get the list of current date folders that match the date pattern and are not logged yet
# # #     date_folders = [
# # #         f for f in os.listdir(base_folder) 
# # #         if os.path.isdir(os.path.join(base_folder, f)) and match_date_pattern(f) 
# # #         and f not in logged_folders  # Only include new folders
# # #     ]
    
# # #     # If there are no new folders, print a message and return
# # #     if not date_folders:
# # #         print("No new folders to scan.")
# # #         return

# # #     # Use ThreadPoolExecutor for parallel folder scanning
# # #     with concurrent.futures.ThreadPoolExecutor() as executor:
# # #         # Run scan_folder in parallel for each new date folder
# # #         log_data = list(executor.map(scan_folder, date_folders, [base_folder] * len(date_folders), [failed_paths] * len(date_folders), [config] * len(date_folders)))

# # #     # Convert the results into a DataFrame
# # #     df = pd.json_normalize(log_data, 'rec_files_data', ['date_folder', 'calib_files'])

# # #     # Dynamically ensure all relevant columns are strings based on config
# # #     status_columns = list(config.keys())
# # #     df[status_columns] = df[status_columns].astype(str)

# # #     # Create pyarrow Table and save as Parquet
# # #     table = pa.Table.from_pandas(df)
# # #     pq.write_table(table, parquet_file)

# # def log_folder_to_parquet(base_folder, parquet_file, failed_paths_file, config):
# #     # Read manually inputted failed paths
# #     failed_paths = read_failed_paths(failed_paths_file)

# #     # Initialize logged folders
# #     logged_folders = []

# #     # Check if the Parquet file exists
# #     if os.path.exists(parquet_file):
# #         # Read the existing log to get already processed date folders
# #         existing_df = pq.read_table(parquet_file).to_pandas()
# #         logged_folders = existing_df['date_folder'].unique() if not existing_df.empty else []
# #     else:
# #         print("No existing Parquet file found. Running full scan.")

# #     # Get the list of current date folders that match the date pattern
# #     date_folders = [
# #         f for f in os.listdir(base_folder) 
# #         if os.path.isdir(os.path.join(base_folder, f)) and match_date_pattern(f)
# #     ]
    
# #     # Filter for new folders not logged yet
# #     new_folders = [f for f in date_folders if f not in logged_folders]

# #     # If no new folders are found, print a message and return
# #     if not new_folders:
# #         print("No new folders to scan.")
# #         return

# #     # Use ThreadPoolExecutor for parallel folder scanning
# #     with concurrent.futures.ThreadPoolExecutor() as executor:
# #         # Run scan_folder in parallel for each new date folder
# #         log_data = list(executor.map(scan_folder, new_folders, [base_folder] * len(new_folders), [failed_paths] * len(new_folders), [config] * len(new_folders)))

# #     # Convert the results into a DataFrame
# #     df = pd.json_normalize(log_data, 'rec_files_data', ['date_folder', 'calib_files'])

# #     # Dynamically ensure all relevant columns are strings based on config
# #     status_columns = list(config.keys())
# #     df[status_columns] = df[status_columns].astype(str)

# #     # Create pyarrow Table and save as Parquet
# #     table = pa.Table.from_pandas(df)
# #     pq.write_table(table, parquet_file)
# #     print("all scannning done")






# # if __name__ == "__main__":
# #     base_folder = "/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ"  # Replace with your base folder
# #     save_path = os.path.join(base_folder, 'paret')
# #     failed_paths_file = '/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/mir_bundle_run/synced_folders/240914_failed_sum_test.txt'  # File containing failed paths

# #     if not os.path.exists(save_path):
# #         os.makedirs(save_path)

# #     parquet_file = os.path.join(save_path, "folder_log_encoded_numb_paralle_test_3.parquet")  # Output Parquet file

# #     # Run the full scan with parallel processing
# #     log_folder_to_parquet(base_folder, parquet_file, failed_paths_file, STATUS_FIELDS_CONFIG)

In [24]:

from utlis.sync_utlis.sync_df_utlis import find_calib_file
from utlis.scan_engine_utlis.scan_engine_utlis import (
    read_failed_paths,
    match_date_pattern,
    assign_status_codes,
)
from scan_engine.status_fields_config import STATUS_FIELDS_CONFIG

# Import functions from utils.py
from utlis.scan_engine_utlis.scan_log_utlis import (
    load_scan_log,
    save_scan_log,
    clean_scan_log,
    update_scan_log,
    get_folders_to_scan
)

def scan_folder(folder_name, base_folder, failed_paths, config, rec_files_to_scan):
    folder_path = os.path.join(base_folder, folder_name)
    rec_files_data = []  # To store rec files and their status
    calib_files = []  # To store calibration files

    # Check for calibration files starting with 'calib'
    for file_name in os.listdir(folder_path):
        if file_name.startswith("calib"):
            calib_files.append(file_name)

    # Traverse subfolders within this folder
    for subfolder_name in rec_files_to_scan:
        subfolder_path = os.path.join(folder_path, subfolder_name)

        # Check for subfolders starting with a digit (rec folders)
        if os.path.isdir(subfolder_path) and subfolder_name[0].isdigit():
            # Find calibration file for each subfolder
            calib_file = find_calib_file(subfolder_path)

            # Assign status codes dynamically based on the config
            rec_file_data = assign_status_codes(
                folder_name, subfolder_path, calib_file, failed_paths, config
            )

            rec_file_data['rec_file'] = subfolder_name  # Add rec_file to the data
            # Add date-time for update and some future
            rec_file_data['scan_time'] = datetime.datetime.now().isoformat()

            rec_files_data.append(rec_file_data)

    return {
        'date_folder': folder_name,
        'calib_files': calib_files,  # Store the calibration files under date_folder level
        'rec_files_data': rec_files_data  # Each rec file with its status fields
    }

def log_folder_to_parquet_sep(base_folder, failed_paths_file, config, force_rescan_rec_files=None, rescan_threshold_days=7):
    """Log folders and save Parquet in subfolders with partial scan support."""

    # Paths for scan log
    scan_log_path = os.path.join(base_folder, 'paret', 'scan_log.csv')

    # Load or initialize the scan log
    scan_log_df = load_scan_log(scan_log_path)

    # Read manually inputted failed paths
    failed_paths = read_failed_paths(failed_paths_file) if failed_paths_file else set()

    # Forced rescans
    # force_rescan_rec_files = [
    #     # ('2023-10-01', '001'),
    #     # ('2023-10-02', '002'),
    #     # Add more as needed
    # ]
    # force_rescan_rec_files_set = set(force_rescan_rec_files)
    
    if force_rescan_rec_files is None:
        force_rescan_rec_files = []
    force_rescan_rec_files_set = set(force_rescan_rec_files)



    # Rescan threshold
    # rescan_threshold_days = 7

    # Determine folders to scan
    folders_to_scan = get_folders_to_scan(base_folder, scan_log_df, rescan_threshold_days, force_rescan_rec_files_set)

    if not folders_to_scan:
        print("No new or modified folders to scan.")
        return

    # Use ThreadPoolExecutor for parallel folder scanning
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for date_folder, rec_files_to_scan in folders_to_scan.items():
            futures.append(
                executor.submit(
                    scan_folder,
                    date_folder,
                    base_folder,
                    failed_paths,
                    config,
                    rec_files_to_scan
                )
            )

        for future in concurrent.futures.as_completed(futures):
            folder_log = future.result()
            date_folder = folder_log['date_folder']
            calib_files = folder_log.get('calib_files', [])

            # Ensure 'calib_files' is always a list of strings
            calib_files = [str(f) for f in calib_files] if calib_files else []

            # Process and save each experiment's log separately
            for rec_file_data in folder_log['rec_files_data']:
                rec_file = rec_file_data['rec_file']
                subfolder_save_path = os.path.join(base_folder, date_folder, rec_file, "folder_log.parquet")

                # Ensure the experiment/rec_file folder exists
                os.makedirs(os.path.dirname(subfolder_save_path), exist_ok=True)

                # Add 'date_folder' and 'calib_files' to rec_file_data
                rec_file_data['date_folder'] = date_folder
                rec_file_data['calib_files'] = calib_files

                # Dynamically ensure all relevant columns are strings based on config
                status_columns = list(config.keys())
                df = pd.DataFrame([rec_file_data])
                df[status_columns] = df[status_columns].astype(str)

                # Convert the data into a DataFrame and save the Parquet file
                table = pa.Table.from_pandas(df)
                pq.write_table(table, subfolder_save_path)

                print(f"Log for {rec_file} saved at {subfolder_save_path}")

                # Update the scan log
                scan_log_df = update_scan_log(scan_log_df, date_folder, rec_file)

    # Clean up the scan log
    scan_log_df = clean_scan_log(scan_log_df, base_folder)

    # Save the updated scan log
    save_scan_log(scan_log_df, scan_log_path)

if __name__ == "__main__":
    base_folder = "/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ"  # Replace with your base folder
    # save_path = os.path.join(base_folder, 'paret')
    failed_paths_file = '/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/mir_bundle_run/synced_folders/240914_failed_sum_test.txt'  # File containing failed paths


    force_rescan_rec_files = [
        # ('2023-10-01', '001'),
        # ('2023-10-02', '002'),
        # Add more as needed
    ]
    rescan_threshold_days = 7 # 7 days, but guess if i mess up i can just change it to automatically rescan all, smile... #0.1

    log_folder_to_parquet_sep(base_folder, failed_paths_file, STATUS_FIELDS_CONFIG,
                              force_rescan_rec_files=force_rescan_rec_files,
                              rescan_threshold_days=rescan_threshold_days)


Log for 20240717_PMC_r1_11_50 saved at /hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_08_16/20240717_PMC_r1_11_50/folder_log.parquet
Log for 20240717_PMC_r2_11_00 saved at /hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_08_16/20240717_PMC_r2_11_00/folder_log.parquet
Log for 1686940_left_right saved at /hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_06_25/1686940_left_right/folder_log.parquet
Log for 1686940_left_right_2 saved at /hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_06_25/1686940_left_right_2/folder_log.parquet
Log for 20240628_PMC_r1_11_43 saved at /hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_08_08/20240628_PMC_r1_11_43/folder_log.parquet
Log for 20240628_PMC_r1_11_26 saved at /hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_08_08/20240628_PMC_r1_11_26/folder_log.parquet
Log for 20240702_PMC_r1_12_02 saved at /hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_08_08/20240702_PMC_r1_12_02/folder_log.parquet
Log for 20240717_PMCr1 saved at /hp

In [25]:
from utlis.scan_engine_utlis.scan_engine_utlis import read_all_parquet_files_auto_exclude
base_folder = "/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ"
"""actually can use read_all_parquet_files in cluster probably, i do not have calib folders locally so need to handle this hhh"""

# Example usage
# base_folder = "/path/to/your/base_folder"  # Replace with your base folder
combined_df = read_all_parquet_files_auto_exclude(base_folder, exclude_columns = ['calib_files']
)

# Display or use the combined DataFrame
print(combined_df)

pyarrow.Table
mir_generate_param: string
sync: string
z_adjusted: string
rec_file: string
scan_time: string
date_folder: string
----
mir_generate_param: [["0"],["0"],...,["0"],["1"]]
sync: [["3"],["3"],...,["0"],["0"]]
z_adjusted: [["0"],["0"],...,["2"],["2"]]
rec_file: [["1686940_left_right"],["1686940_left_right_2"],...,["20240916v1r1_18_53"],["20240916v1r1_19_18"]]
scan_time: [["2024-10-15T16:51:45.803150"],["2024-10-15T16:51:45.806128"],...,["2024-10-15T16:51:45.942541"],["2024-10-15T16:51:45.894511"]]
date_folder: [["2024_06_25"],["2024_06_25"],...,["2024_10_07"],["2024_10_07"]]


In [12]:
# import pyarrow.compute as pc

# # Define a mask to identify rows with 'False' or null values in specific columns
# mask_false_mir = pc.equal(table['mir_generate_param'], 'False')
# mask_false_sync = pc.equal(table['sync'], 'False')
# mask_false_z_adjusted = pc.equal(table['z_adjusted'], 'False')

# mask_null_mir = pc.is_null(table['mir_generate_param'])
# mask_null_sync = pc.is_null(table['sync'])
# mask_null_z_adjusted = pc.is_null(table['z_adjusted'])

# # Combine the masks using multiple pc.or_() calls
# mask = pc.or_(
#     pc.or_(mask_false_mir, mask_false_sync),
#     pc.or_(mask_false_z_adjusted, 
#            pc.or_(mask_null_mir, 
#                   pc.or_(mask_null_sync, mask_null_z_adjusted)))
# )

# # Apply the mask to filter the rows
# filtered_table = table.filter(mask)

# # Print the filtered rows for inspection
# print(filtered_table.to_pandas())  # Convert to pandas for easier viewing in Jupyter Notebook


    mir_generate_param    sync z_adjusted  \
0                False   False      False   
1                False  FAILED      False   
2                False  FAILED      False   
3                False   False      False   
4                False   False      False   
..                 ...     ...        ...   
101              False  FAILED      False   
102              False   False      False   
103              False   False      False   
104              False   False      False   
105              False   False      False   

                                rec_file scan_time date_folder  
0         1691486_left_right_habituation      None  2024_07_03  
1               1691486_left_habituation      None  2024_07_03  
2             1691486_left_caffeine_1050      None  2024_07_03  
3       1691486_left_right_caffeine_1448      None  2024_07_03  
4             1691486_left_caffeine_1051      None  2024_07_03  
..                                   ...       ...         ...  
101 

In [26]:
import pyarrow.compute as pc
table = combined_df
# Filter mir_generate_param == 0 and sync != 3
filter_mask = pc.and_(
    pc.equal(table['mir_generate_param'], '0'),
    pc.not_equal(table['sync'], '3')
)

# Apply the filter and print the results
filtered_table = table.filter(filter_mask)

# Print each row of the filtered table
print(filtered_table.to_pandas())  # This will display the filtered data in a familiar pandas-like format


  mir_generate_param sync z_adjusted            rec_file  \
0                  0    0          2        20240819V1r1   
1                  0    0          2   241004_miniscopes   
2                  0    0          2  20240916v1r1_18_53   

                    scan_time date_folder  
0  2024-10-15T16:51:45.975507  2024_10_03  
1  2024-10-15T16:51:45.895409  2024_10_04  
2  2024-10-15T16:51:45.942541  2024_10_07  


In [19]:
filtered_table

pyarrow.Table
mir_generate_param: string
sync: string
z_adjusted: string
rec_file: string
scan_time: string
date_folder: string
----
mir_generate_param: [["0"],["0"],...,["0"],["0"]]
sync: [["0"],["0"],...,["0"],["0"]]
z_adjusted: [["0"],["0"],...,["2"],["2"]]
rec_file: [["20240717_PMC_r1_11_50"],["20240717_PMC_r2_11_00"],...,["20240916v1r1_18_53"],["20240916v1r1_19_18"]]
scan_time: [["2024-10-15T15:47:18.408211"],["2024-10-15T15:47:18.497773"],...,["2024-10-15T15:47:18.700934"],["2024-10-15T15:47:18.498807"]]
date_folder: [["2024_08_16"],["2024_08_16"],...,["2024_10_07"],["2024_10_07"]]

In [20]:
# too lazy to write general/reuseable codes, thus
# below is only to process mir_generate_param

import os
import concurrent.futures
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import datetime
sys.path.append(os.path.abspath('../..'))
from utlis.exe_engine_utlis.mir_generate_param_modu import mir_generate_param_z

# Function to process each "unit" (rec_file) and update its status in the corresponding Parquet file
def process_unit_and_update_status(rec_file_data, base_folder):
    date_folder = rec_file_data['date_folder']
    rec_file = rec_file_data['rec_file']
    
    # Generate the paths needed for processing
    combined_path = os.path.join(base_folder, date_folder)
    calib_path = rec_file_data['calib_path'] if 'calib_path' in rec_file_data else os.path.join(base_folder, 'calib_before')
    
    if not calib_path:  # Check for empty or None calib_path
        print(f'No calib folder found. Aborting. {combined_path}/{rec_file}')
        return
    
    output_file = f'{os.path.basename(date_folder)}_{rec_file}_{os.path.basename(calib_path)}_label3d_dannce.mat'

    # Call your processing function
    mir_generate_param_z(combined_path, calib_path, rec_file, output_file)
    print("mir_generate_param ran successfully.")

    # After processing, update the status in the specific Parquet file
    parquet_file_path = os.path.join(base_folder, date_folder, rec_file, "folder_log.parquet")

    # Load the existing Parquet file
    try:
        table = pq.read_table(parquet_file_path)
        df = table.to_pandas()  # Convert to pandas for easier manipulation
    except FileNotFoundError:
        print(f"Parquet file not found at {parquet_file_path}")
        return

    # Update the status field (assuming 'sync' is the column)
    df['mir_generate_param'] = '1'  # Set status to '1' for processed

    # Add scan_time (or other updates)
    df['scan_time'] = datetime.datetime.now().isoformat()

    # Write the updated DataFrame back to the Parquet file
    updated_table = pa.Table.from_pandas(df)
    pq.write_table(updated_table, parquet_file_path)

    print(f"Updated Parquet file at {parquet_file_path} with new status.")

# Function to handle parallel processing and status updates
def parallel_process_and_update(filtered_table, base_folder):
    # Convert PyArrow table to pandas DataFrame
    filtered_df = filtered_table.to_pandas()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(process_unit_and_update_status, row._asdict(), base_folder)
            for row in filtered_df.itertuples(index=False)
        ]
        
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # Handle any exceptions
            except Exception as e:
                print(f"Error in processing: {e}")

# Example function call
# filtered_table = [...]  # Your filtered PyArrow table here
# base_folder = '/path/to/your/base_folder'
parallel_process_and_update(filtered_table, base_folder)


Found 0 calibration files.
Found 0 calibration files.
Found 0 calibration files.
Found 0 calibration files.
Found 0 calibration files.
Found 0 calibration files.
Found 0 calibration files.
Found 0 calibration files.
Found 0 calibration files.
Found 0 calibration files.
Found 0 calibration files.
Found 0 calibration files.
Frame count: 27900
Frame count: 27300
Data saved to /hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_08_29/20240717_PMCr2/2024_08_29_20240717_PMCr2_calib_before_label3d_dannce.mat
mir_generate_param ran successfully.
Data saved to /hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_09_18/20240707-PMC-r1/2024_09_18_20240707-PMC-r1_calib_before_label3d_dannce.mat
mir_generate_param ran successfully.
Frame count: 14243
Frame count: 21969
Frame count:Error opening video file.
mir_generate_param ran successfully.
Frame count: 27900
Frame count: 30000
 27300
Data saved to /hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_10_03/20240819V1r2/2024_10_03_20240819V1r2_ca

In [21]:
from utlis.scan_engine_utlis.scan_engine_utlis import read_all_parquet_files_auto_exclude
base_folder = "/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ"
"""actually can use read_all_parquet_files in cluster probably, i do not have calib folders locally so need to handle this hhh"""

# Example usage
# base_folder = "/path/to/your/base_folder"  # Replace with your base folder
combined_df = read_all_parquet_files_auto_exclude(base_folder, exclude_columns = ['calib_files']
)

# Display or use the combined DataFrame
print(combined_df)

pyarrow.Table
mir_generate_param: string
sync: string
z_adjusted: string
rec_file: string
scan_time: string
date_folder: string
----
mir_generate_param: [["0"],["0"],...,["0"],["0"]]
sync: [["3"],["3"],...,["1"],["1"]]
z_adjusted: [["0"],["0"],...,["2"],["2"]]
rec_file: [["1686940_left_right"],["1686940_left_right_2"],...,["20240916v1r1_18_53"],["20240916v1r1_19_18"]]
scan_time: [["2024-10-15T15:47:18.496021"],["2024-10-15T15:47:18.498828"],...,["2024-10-15T16:38:36.504311"],["2024-10-15T16:38:37.212976"]]
date_folder: [["2024_06_25"],["2024_06_25"],...,["2024_10_07"],["2024_10_07"]]


In [22]:
import pyarrow.compute as pc
table = combined_df
# Filter mir_generate_param == 0 and sync != 3
filter_mask = pc.and_(
    pc.equal(table['mir_generate_param'], '0'),
    pc.not_equal(table['sync'], '3'),
    pc.not_equal(table['sync'], '3')
)

# Apply the filter and print the results
filtered_table = table.filter(filter_mask)

# Print each row of the filtered table
print(filtered_table.to_pandas())  # This will display the filtered data in a familiar pandas-like format

   mir_generate_param sync z_adjusted                  rec_file  \
0                   0    1          0     20240717_PMC_r1_11_50   
1                   0    1          0     20240717_PMC_r2_11_00   
2                   0    1          0            20240717_PMCr1   
3                   0    1          0            20240717_PMCr2   
4                   0    1          0            20240730_PMCr2   
5                   0    1          0            20240717_PMCr2   
6                   0    1          2           20240707-PMC-r1   
7                   0    1          2            20240819-V1-r2   
8                   0    1          2            20240819_V1_r1   
9                   0    1          2          20240819_V1_r1_2   
10                  0    1          2              20240819V1r1   
11                  0    1          2              20240819V1r2   
12                  0    1          2              20240819V1r1   
13                  0    1          2        20240819V1r1_20_1

In [ ]:
# too lazy to write general/reuseable codes, thus
# below is only to process mir_generate_param

import os
import concurrent.futures
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import datetime
sys.path.append(os.path.abspath('../..'))
from utlis.exe_engine_utlis.mir_generate_param_modu import mir_generate_param_z

# Function to process each "unit" (rec_file) and update its status in the corresponding Parquet file
def process_unit_and_update_status(rec_file_data, base_folder):
    date_folder = rec_file_data['date_folder']
    rec_file = rec_file_data['rec_file']
    
    # Generate the paths needed for processing
    combined_path = os.path.join(base_folder, date_folder)
    calib_path = rec_file_data['calib_path'] if 'calib_path' in rec_file_data else os.path.join(base_folder, 'calib_before')
    
    if not calib_path:  # Check for empty or None calib_path
        print(f'No calib folder found. Aborting. {combined_path}/{rec_file}')
        return
    
    output_file = f'{os.path.basename(date_folder)}_{rec_file}_{os.path.basename(calib_path)}_label3d_dannce.mat'

    # Call your processing function
    mir_generate_param_z(combined_path, calib_path, rec_file, output_file)
    print("mir_generate_param ran successfully.")

    # After processing, update the status in the specific Parquet file
    parquet_file_path = os.path.join(base_folder, date_folder, rec_file, "folder_log.parquet")

    # Load the existing Parquet file
    try:
        table = pq.read_table(parquet_file_path)
        df = table.to_pandas()  # Convert to pandas for easier manipulation
    except FileNotFoundError:
        print(f"Parquet file not found at {parquet_file_path}")
        return

    # Update the status field (assuming 'sync' is the column)
    df['mir_generate_param'] = '1'  # Set status to '1' for processed

    # Add scan_time (or other updates)
    df['scan_time'] = datetime.datetime.now().isoformat()

    # Write the updated DataFrame back to the Parquet file
    updated_table = pa.Table.from_pandas(df)
    pq.write_table(updated_table, parquet_file_path)

    print(f"Updated Parquet file at {parquet_file_path} with new status.")

# Function to handle parallel processing and status updates
def parallel_process_and_update(filtered_table, base_folder):
    # Convert PyArrow table to pandas DataFrame
    filtered_df = filtered_table.to_pandas()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(process_unit_and_update_status, row._asdict(), base_folder)
            for row in filtered_df.itertuples(index=False)
        ]
        
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # Handle any exceptions
            except Exception as e:
                print(f"Error in processing: {e}")

# Example function call
# filtered_table = [...]  # Your filtered PyArrow table here
# base_folder = '/path/to/your/base_folder'
parallel_process_and_update(filtered_table, base_folder)
